In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Cargar el dataset Oxford Flowers 102
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True, split=['train', 'test'])

# Preprocesar las imágenes: redimensionar y normalizar
IMG_SIZE = 224
def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0 #Se normalizan los valores de los píxeles dividiéndolos por 255 
                                                                 #para llevarlos al rango [0, 1], lo que ayuda a que el modelo entrene de manera más eficiente.
    return image, label

train_dataset = dataset[0].map(format_image)
test_dataset = dataset[1].map(format_image)

# Batching y shuffling (Para mejorar la eficiencia durante el entrenamiento, las imágenes se agrupan en batches de tamaño 32)
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(1024).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE) (#se aplica shuffling para asegurar que el modelo no aprenda patrones basados en el orden de las imágenes.)

# Cargar la red preentrenada MobileNetV2 sin la capa superior
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Congelar las capas de la red preentrenada
base_model.trainable = False

# Añadir nuevas capas densas para la tarea de clasificación multiclase
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(102, activation='softmax')  # Clasificación multiclase (102 clases)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 3/3 [02:19<00:00, 46.59s/ url]


Dataset oxford_flowers102 downloaded and prepared to C:\Users\diego\tensorflow_datasets\oxford_flowers102\2.1.1. Subsequent calls will reuse this data.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 115s 4s/step - accuracy: 0.0353 - loss: 4.6035 - val_accuracy: 0.1691 - val_loss: 3.9751
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.3598 - loss: 3.3596 - val_accuracy: 0.4064 - val_loss: 2.7836
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - accuracy: 0.6332 - loss: 2.0132 - val_accuracy: 0.5786 - val_loss: 1.9955
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.8447 - loss: 1.0275 - val_accuracy: 0.6357 - val_loss: 1.6345
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 118s 4s/step - accuracy: 0.9397 - loss: 0.6086 - val_accuracy: 0.6673 - val_loss: 1.4361
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 117s 4s/step - accuracy: 0.9549 - loss: 0.4228 - val_accuracy: 0.6962 - val_loss: 1.2664
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━

# Introducción

En este caso utilizaremos Transfer Learning para clasificar imágenes del dataset Oxford Flowers 102, que contiene más de 7,000 imágenes de 102 especies diferentes de flores. La tarea consiste en predecir a qué categoría pertenece cada imagen de flor, lo cual es un reto debido a la variedad visual entre clases.

Para abordar esta tarea, emplearemos el modelo preentrenado MobileNetV2, que ha sido entrenado previamente en el dataset ImageNet con millones de imágenes. Aprovecharemos las características visuales que este modelo ya ha aprendido, lo que nos permitirá mejorar la precisión y reducir el tiempo de entrenamiento.

Se escogió el dataset Oxford Flowers 102 porque ofrece un desafío interesante para la clasificación de imágenes debido a su gran variedad visual con 102 categorías diferentes de flores. Esto lo hace ideal para aplicar Transfer Learning, ya que podemos aprovechar las características aprendidas por modelos preentrenados para mejorar la precisión en un conjunto de datos específico y complejo. Además, es un dataset bien etiquetado y reconocido en tareas de visión por computadora, lo que facilita la evaluación y comparación de resultados.

# Exploración, Explicación y Limpieza de Datos

1. Explicación de dónde se obtuvo el dataset y su contexto:

El dataset Oxford Flowers 102 se obtuvo a través de la librería TensorFlow Datasets. Fue creado por la Universidad de Oxford y contiene imágenes de 102 especies de flores. Este dataset es ampliamente utilizado en tareas de clasificación de imágenes multiclase debido a la gran variabilidad visual que presentan las flores, lo que lo convierte en un buen desafío para modelos de visión por computadora.

El dataset tiene más de 7,000 imágenes, y las clases están desbalanceadas, es decir, algunas clases tienen más imágenes que otras. Esto plantea un reto adicional en el proceso de entrenamiento del modelo, pero puede manejarse adecuadamente con técnicas de Transfer Learning.

2. Análisis del dataset:

Número de instancias: El dataset contiene 7,189 imágenes en total, divididas en 102 clases.

Número de columnas: Cada imagen está acompañada de una etiqueta (label) que indica a qué especie de flor pertenece, por lo que el dataset tiene dos columnas principales: la imagen (input) y la etiqueta (output).

Tipos de datos:
    Las imágenes son arrays de píxeles con 3 canales (RGB), que deben ser redimensionadas para que tengan un tamaño uniforme.
    Las etiquetas son valores enteros que representan las 102 clases de flores. Estas etiquetas están codificadas numéricamente del 0 al 101.

# Resultados e Interpretación
1. Resultados del modelo

Después de entrenar el modelo durante 10 épocas utilizando Transfer Learning con MobileNetV2, los resultados son los siguientes:

    Precisión en el conjunto de entrenamiento (Epoch 10): 99.97%
    Pérdida en el conjunto de entrenamiento (Epoch 10): 0.1005
    Precisión en el conjunto de validación (Epoch 10): 73.25%
    Pérdida en el conjunto de validación (Epoch 10): 1.0782

2. Interpretación de los resultados

    Rendimiento en entrenamiento: El modelo alcanzó una precisión muy alta (99.97%) en el conjunto de entrenamiento, lo que indica que ha aprendido a clasificar correctamente las imágenes de flores con bastante exactitud.

    Rendimiento en validación: La precisión en el conjunto de validación es más baja (73.25%), lo que es normal dado que el conjunto de validación contiene datos no vistos previamente. Esto indica que el modelo ha generalizado bien, aunque existe una diferencia considerable entre entrenamiento y validación. Esta diferencia podría ser señal de un posible sobreajuste.

3. Análisis del sobreajuste

    La alta precisión en el entrenamiento (casi el 100%) y la menor precisión en el conjunto de validación (73.25%) sugieren que el modelo podría estar sobreajustándose a los datos de entrenamiento.
    Sin embargo, la mejora continua en la precisión de validación y la disminución constante de la pérdida sugieren que el modelo sigue aprendiendo, y más epochs de entrenamiento podrían mejorar el rendimiento en validación.

4. Posibles mejoras

    Descongelar algunas capas del modelo preentrenado: Para mejorar la precisión en el conjunto de validación, se podría intentar descongelar algunas de las capas superiores de MobileNetV2 para permitir un ajuste más fino de los pesos en las últimas capas convolucionales. Esto permitiría al modelo aprender características más específicas de las flores.

    Aumentar el número de epochs: Continuar entrenando el modelo más allá de las 10 epochs puede mejorar la precisión en el conjunto de validación, ya que el modelo parece seguir mejorando con cada epoch.

    Técnicas de regularización: Para mitigar el sobreajuste, se podrían añadir técnicas de regularización como Dropout o Early Stopping para detener el entrenamiento cuando la precisión de validación deje de mejorar.

En resumen, los resultados muestran un buen rendimiento general del modelo, con alta precisión en el conjunto de entrenamiento y una precisión aceptable en el conjunto de validación. 

# Conclusión

En este notebook, aplicamos Transfer Learning con MobileNetV2 para clasificar imágenes del dataset Oxford Flowers 102 en 102 categorías. El modelo logró una alta precisión del 99.97% en el entrenamiento y un 73.25% en validación, mostrando buena generalización pero con indicios de sobreajuste.

Transfer Learning permitió reducir el tiempo de entrenamiento y obtener buenos resultados. Para mejorar el rendimiento, sería útil descongelar capas adicionales y aplicar regularización. En general, el objetivo de clasificar imágenes de flores fue cumplido con éxito.